### Evaluating CPM with CEBaB
Our Causal Proxy Model (CPM) is for providing concept-based explanation for a blackbox model. We use newly developed CEBaB benchmark for comparing CPM with other concept-based explanation methods. This notebook evaluates CPM with CEBaB benchmark under different settings.

More importantly, we introduce new baselines for CPM as well. Formally, we evaluate the blackbox model with interchange intervention evaluation (which will be introduced in details below).

In this notebook, we can evaluate the following models:
- CPM: `BERT-base-uncased`
- CPM: `RoBERTa-base`
- CPM: `GPT2`
- CPM: `LSTM+GloVe`
- CPM: `Control`

and we can evaluate with the following conditions:
- 2-class
- 3-class
- 5-class

#### Imports and Libs

In [1]:
from libs import *
from modelings.modelings_bert import *
from modelings.modelings_roberta import *
from modelings.modelings_gpt2 import *
from modelings.modelings_lstm import *
"""
For evaluate, we use a single random seed, as
the models are trained with 5 different seeds
already.
"""
_ = random.seed(123)
_ = np.random.seed(123)
_ = torch.manual_seed(123)

#### Main evaluate script

In [22]:
results = {}
# rerunning the boxes below will only append stuffs to the results.

In [23]:
"""
The following blocks will run CEBaB benchmark in
all the combinations of the following conditions.
"""
grid = {
    "eval_split": ["test"],
    # dev,test
    "control": ["approximate"],
    # baseline-random,baseline-blackbox,hdims,layers,ks,approximate,ablation
    "seed": [42, 66, 77],
    # 42, 66, 77
    "h_dim": [192],
    # 1,16,64,128,192
    # 1,16,64,75
    "interchange_layer" : [10],
    # 0,1; 2,4,6,8,10,12
    "class_num": [5],
    "k" : [0], 
    # 0,10,100,500,1000,3000,6000,9848,19684
    "alpha" : [1.0],
    # 0.0,1.0
    "beta" : [1.0],
    # 0.0,1.0
    "gemma" : [3.0],
    # 0.0,3.0
    "model_arch" : ["bert-base-uncased"],
    # lstm, bert-base-uncased, roberta-base, gpt2
    "lr" : ["8e-05"],
    # 8e-05; 0.001
    "counterfactual_type" : ["approximate"]
    # approximate,true
}

keys, values = zip(*grid.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

device = 'cuda:9'
batch_size = 32

In [24]:
for i in range(len(permutations_dicts)):
    
    eval_split=permutations_dicts[i]["eval_split"]
    seed=permutations_dicts[i]["seed"]
    class_num=permutations_dicts[i]["class_num"]
    alpha=permutations_dicts[i]["alpha"]
    beta=permutations_dicts[i]["beta"]
    gemma=permutations_dicts[i]["gemma"]
    h_dim=permutations_dicts[i]["h_dim"]
    dataset_type = f'{class_num}-way'
    control=permutations_dicts[i]["control"]
    model_arch=permutations_dicts[i]["model_arch"]
    k=permutations_dicts[i]["k"]
    interchange_layer=permutations_dicts[i]["interchange_layer"]
    lr=permutations_dicts[i]["lr"]
    counterfactual_type=permutations_dicts[i]["counterfactual_type"]
    
    if model_arch == "bert-base-uncased":
        model_path = "BERT"
        model_module = BERTForCEBaB
        explainer_module = CausalProxyModelForBERT
    elif model_arch == "roberta-base":
        model_path = "RoBERTa" 
        model_module = RoBERTaForCEBaB
        explainer_module = CausalProxyModelForRoBERTa
    elif model_arch == "gpt2":
        model_path = "gpt2"
        model_module = GPT2ForCEBaB
        explainer_module = CausalProxyModelForGPT2
    elif model_arch == "lstm":
        model_path = "lstm"
        model_module = LSTMForCEBaB
        explainer_module = CausalProxyModelForLSTM
    model_path += f"-{control}"
    grid_conditions=(
        ("eval_split", eval_split),
        ("control", control),
        ("seed", seed),
        ("h_dim", h_dim),
        ("interchange_layer", interchange_layer),
        ("class_num", class_num),
        ("k", k),
        ("alpha", alpha),
        ("beta", beta),
        ("gemma", gemma),
        ("model_arch", model_arch),
        ("lr", lr),
        ("counterfactual_type", counterfactual_type)
    )
    print("Running for this setting: ", grid_conditions)

    blackbox_model_path = f'CEBaB/{model_arch}.CEBaB.sa.'\
                          f'{class_num}-class.exclusive.seed_{seed}'
    cpm_model_path = f'../proxy_training_results/{model_path}/'\
                     f'cebab.alpha.{alpha}.beta.{beta}.gemma.{gemma}.'\
                     f'lr.{lr}.dim.{h_dim}.hightype.{model_arch}.'\
                     f'CEBaB.cls.dropout.0.1.enc.dropout.0.1.counter.type.'\
                     f'{counterfactual_type}.k.{k}.int.layer.{interchange_layer}.'\
                     f'seed_{seed}/'

    # load data from HF
    cebab = datasets.load_dataset(
        'CEBaB/CEBaB', use_auth_token=True,
        cache_dir="../train_cache/"
    )

    train, dev, test = preprocess_hf_dataset_inclusive(
        cebab, verbose=1, dataset_type=dataset_type
    )

    eval_dataset = dev if eval_split == 'dev' else test

    tf_model = model_module(
        blackbox_model_path, 
        device=device, 
        batch_size=batch_size
    )
    explainer = explainer_module(
        blackbox_model_path,
        cpm_model_path, 
        device=device, 
        batch_size=batch_size,
        intervention_h_dim=h_dim,
    )

    result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
    CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
    ACaCE_per_aspect, performance_report = cebab_pipeline(
        tf_model, explainer, 
        train, eval_dataset,
        seed, k, dataset_type=dataset_type, 
        shorten_model_name=False, 
        train_setting="inclusive", 
        approximate=False if counterfactual_type == "true" else True
    )
    
    results[grid_conditions] = (
        result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
        CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
        ACaCE_per_aspect, performance_report
    )


Running for this setting:  (('eval_split', 'test'), ('control', 'approximate'), ('seed', 42), ('h_dim', 192), ('interchange_layer', 10), ('class_num', 5), ('k', 0), ('alpha', 1.0), ('beta', 1.0), ('gemma', 3.0), ('model_arch', 'bert-base-uncased'), ('lr', '8e-05'), ('counterfactual_type', 'approximate'))


Using custom data configuration CEBaB--CEBaB-ccd674d249652bd4
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-ccd674d249652bd4/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/5 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train_exclusive dataset.
Dropping no majority reviews: 16.03% of train_inclusive dataset.
intervention_h_dim=192


../eval_pipeline/utils/data_utils.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs['intervention_type'][pairs[f'{aspect}_aspect_majority_base'] != pairs[f'{aspect}_aspect_majority_counterfactual']] = aspect
  0%|          | 0/124 [00:00<?, ?it/s]/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/site-packages/transformers/modeling_utils.py:714: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning
100%|██████████| 124/124 [00:17<00:00,  7.23it/s]


...

Running for this setting:  (('eval_split', 'test'), ('control', 'approximate'), ('seed', 77), ('h_dim', 192), ('interchange_layer', 10), ('class_num', 5), ('k', 0), ('alpha', 1.0), ('beta', 1.0), ('gemma', 3.0), ('model_arch', 'bert-base-uncased'), ('lr', '8e-05'), ('counterfactual_type', 'approximate'))


Using custom data configuration CEBaB--CEBaB-ccd674d249652bd4
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-ccd674d249652bd4/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/5 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train_exclusive dataset.
Dropping no majority reviews: 16.03% of train_inclusive dataset.
intervention_h_dim=192


100%|██████████| 124/124 [00:17<00:00,  7.25it/s]


#### Show your results

In [17]:
important_keys = [
    "eval_split",
    "control", "seed", 
    "h_dim", "interchange_layer", 
    "class_num", "k", 
    "beta", "gemma", 
    "model_arch", "lr", "counterfactual_type"
]
values = []
for k, v in results.items():
    _values = []
    for ik in important_keys:
        _values.append(dict(k)[ik])
    _values.append(v[2]["ICaCE-L2"].iloc[0])
    _values.append(v[2]["ICaCE-cosine"].iloc[0])
    _values.append(v[2]["ICaCE-normdiff"].iloc[0])
    _values.append(v[-1].iloc[0][0])
    values.append(_values)
important_keys.extend(["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"])
df = pd.DataFrame(values, columns=important_keys)
df.sort_values(by=['seed', 'interchange_layer', 'h_dim'], ascending=True)

,eval_split,control,seed,h_dim,interchange_layer,class_num,k,beta,gemma,model_arch,lr,counterfactual_type,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,test,ks,42,192,10,5,10,1.0,3.0,bert-base-uncased,8e-05,true,0.8313,0.6325,0.5932,0.463772
1,test,ks,42,192,10,5,100,1.0,3.0,bert-base-uncased,8e-05,true,0.7766,0.5422,0.6301,0.602434
2,test,ks,42,192,10,5,500,1.0,3.0,bert-base-uncased,8e-05,true,0.7505,0.5187,0.6145,0.621132
3,test,ks,42,192,10,5,1000,1.0,3.0,bert-base-uncased,8e-05,true,0.7037,0.5135,0.5924,0.641056
4,test,ks,42,192,10,5,3000,1.0,3.0,bert-base-uncased,8e-05,true,0.4768,0.3923,0.2897,0.706076
5,test,ks,42,192,10,5,6000,1.0,3.0,bert-base-uncased,8e-05,true,0.4583,0.3759,0.2674,0.707092
6,test,ks,42,192,10,5,9848,1.0,3.0,bert-base-uncased,8e-05,true,0.4655,0.3724,0.2906,0.695415
7,test,ks,42,192,10,5,19684,1.0,3.0,bert-base-uncased,8e-05,true,0.4295,0.3520,0.2547,0.699719
8,test,ks,66,192,10,5,10,1.0,3.0,bert-base-uncased,8e-05,true,0.9225,0.6711,0.5690,0.245234
9,test,ks,66,192,10,5,100,1.0,3.0,bert-base-uncased,8e-05,true,0.7494,0.5711,0.6197,0.659658


#### Aggregating results in the way you like

In [18]:
groupby_key = "k"

model_arch = grid["model_arch"][0]
control = grid["control"][0]
df.groupby([groupby_key], as_index=False).mean().to_csv(
    f"./csv-results/{model_arch}-{control}-mean.csv"
)

In [19]:
df.groupby([groupby_key], as_index=False).std().to_csv(
    f"./csv-results/{model_arch}-{control}-std.csv"
)

In [20]:
df.groupby([groupby_key], as_index=False).mean()

,k,seed,h_dim,interchange_layer,class_num,beta,gemma,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,10,61.666667,192.0,10.0,5.0,1.0,3.0,0.880000,0.665800,0.580300,0.361253
1,100,61.666667,192.0,10.0,5.0,1.0,3.0,0.768267,0.562500,0.632533,0.626145
2,500,61.666667,192.0,10.0,5.0,1.0,3.0,0.751233,0.536800,0.631633,0.651780
3,1000,61.666667,192.0,10.0,5.0,1.0,3.0,0.727467,0.531933,0.608633,0.637735
4,3000,61.666667,192.0,10.0,5.0,1.0,3.0,0.501167,0.400767,0.317600,0.702252
5,6000,61.666667,192.0,10.0,5.0,1.0,3.0,0.465900,0.378533,0.269067,0.707178
6,9848,61.666667,192.0,10.0,5.0,1.0,3.0,0.466100,0.365467,0.288700,0.700836
7,19684,61.666667,192.0,10.0,5.0,1.0,3.0,0.447833,0.364233,0.270700,0.694854


In [21]:
df.groupby([groupby_key], as_index=False).std()

,k,seed,h_dim,interchange_layer,class_num,beta,gemma,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,10,17.897858,0.0,0.0,0.0,0.0,0.0,0.045915,0.030992,0.012179,0.109893
1,100,17.897858,0.0,0.0,0.0,0.0,0.0,0.016376,0.017649,0.014207,0.029845
2,500,17.897858,0.0,0.0,0.0,0.0,0.0,0.024608,0.015897,0.029503,0.026876
3,1000,17.897858,0.0,0.0,0.0,0.0,0.0,0.030605,0.017371,0.024981,0.014077
4,3000,17.897858,0.0,0.0,0.0,0.0,0.0,0.026272,0.015012,0.035020,0.004957
5,6000,17.897858,0.0,0.0,0.0,0.0,0.0,0.012819,0.006279,0.007638,0.001737
6,9848,17.897858,0.0,0.0,0.0,0.0,0.0,0.026805,0.015879,0.037486,0.009442
7,19684,17.897858,0.0,0.0,0.0,0.0,0.0,0.020712,0.027383,0.040824,0.010259


#### Save your results somewhere and load again to tabularize your results altogether

In [ ]:
output_name = input("Plase give an output file name: ")

output_directory = f'../proxy_training_results/{model_path}/'
output_filename = os.path.join(output_directory, f'{output_name}.pkl')
print("Writing to file: ", output_filename)
with open(output_filename, 'wb') as f:
    pickle.dump(results, f)